In [1]:
import os
%pwd

'c:\\Users\\Public\\Documents\\Project\\Kidney-Diseases-Classification\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\Public\\Documents\\Project\\Kidney-Diseases-Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    validation_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [4]:
from kidney_diseases_classification.constants import *
from kidney_diseases_classification.utils.common import read_yaml, create_directories
import tensorflow as tf

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.root_dir, "dataset","train")
        validation_data = os.path.join(self.config.data_ingestion.root_dir, "dataset","val")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            validation_data=Path(validation_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        ''' 
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.2
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
     '''
        train_datagenerator = ImageDataGenerator(rescale = 1/255.0,
                        rotation_range=15,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range = 0.2,
                        vertical_flip=True,
                        horizontal_flip = True,
                        fill_mode="reflect")
        
        self.train_generator = train_datagenerator.flow_from_directory(
                            directory=self.config.training_data,
                            target_size=self.config.params_image_size[:-1],
                            class_mode='categorical', 
                            batch_size = self.config.params_batch_size)
        
        val_datagen = ImageDataGenerator(rescale = 1/255.0)


        self.valid_generator = train_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            target_size=self.config.params_image_size[:-1],
            class_mode='categorical',
            batch_size = self.config.params_batch_size)
            
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        '''
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )
        '''
        self.model.fit(
                    self.train_generator,
                    steps_per_epoch=len(self.train_generator),
                    epochs=self.config.params_epochs,
                    validation_data=self.valid_generator)

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [8]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2023-10-18 22:57:45,003: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-18 22:57:45,008: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-18 22:57:45,010: INFO: common: created directory at: artifacts]
[2023-10-18 22:57:45,011: INFO: common: created directory at: artifacts\training]
Found 9955 images belonging to 4 classes.
Found 1242 images belonging to 4 classes.
Epoch 1/5
156/156 [==============================] - 390s 2s/step - loss: 2.3918 - accuracy: 0.5625 - val_loss: 0.6465 - val_accuracy: 0.7391
Epoch 2/5
156/156 [==============================] - 333s 2s/step - loss: 0.6681 - accuracy: 0.7415 - val_loss: 0.5058 - val_accuracy: 0.8019
Epoch 3/5
 10/156 [>.............................] - ETA: 4:44 - loss: 0.5468 - accuracy: 0.7672

KeyboardInterrupt: 